In [36]:
import os
import pandas as pd
from sqlalchemy import create_engine

# 1. Build engine
DATABASE_DSN = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:Czheyuan0227@"
    "aws-0-us-east-2.pooler.supabase.com:6543/postgres?sslmode=require"
)
engine = create_engine(DATABASE_DSN, pool_pre_ping=True)

# 2. Read entire table
df = pd.read_sql_table("wo_structured", con=engine, schema="public")

ERP_df= df[['Customer', 'Customer PO', 'WO', 'Product Number', 'Qty', 
                               'Stock_Available', 'ATP(LT)', 'In Stock(Inventory)', 'On Hand', 'On Sales Order', 'On PO', 'Component_Status', 'Lead Time', 'Picked']]



ERP_df.loc[:, "Lead Time"] = pd.to_datetime(ERP_df["Lead Time"]).dt.date

filtered_df = ERP_df[ERP_df['WO'] == 'SO-20251220']

# filtered_df = ERP_df[ERP_df['Customer'] == 'Aerflite Canada Inc.']


# Apply styling
styled = (
    filtered_df.style
    .map(lambda v: 'background-color: red; color: white' if v == 'Shortage' else '', subset='Component_Status')
    .set_table_styles([
            {
                'selector': 'th.col_heading.level0.col' + str(filtered_df.columns.get_loc('Stock_Available')),
                'props': [('background-color', 'orange'), ('color', 'black')]
            }
        ])
        .format(precision=0, na_rep='', thousands=",")
    )

# Display the styled DataFrame
display(styled)

,Customer,Customer PO,WO,Product Number,Qty,Stock_Available,ATP(LT),In Stock(Inventory),On Hand,On Sales Order,On PO,Component_Status,Lead Time,Picked
670,Dataspeed Inc.,PO4985,SO-20251220,Nuvo-8108GC-QD-A6,2,-21,0,14,14,35,28,Shortage,2025-10-17 00:00:00,No
671,Dataspeed Inc.,PO4985,SO-20251220,E-2278GE,2,-6,0,21,33,39,9,Shortage,2025-10-17 00:00:00,No
672,Dataspeed Inc.,PO4985,SO-20251220,GC-RTXA4000-PNY,2,0,1,3,3,3,0,Shortage,2025-10-17 00:00:00,No
673,Dataspeed Inc.,PO4985,SO-20251220,DDR4-8GB-WT32-SM,8,31,31,104,112,81,0,Available,2025-10-17 00:00:00,No
674,Dataspeed Inc.,PO4985,SO-20251220,M.280-SSD-1TB-PCIe4-TLCWT-IK1,2,52,52,54,55,3,0,Available,2025-10-17 00:00:00,No
675,Dataspeed Inc.,PO4985,SO-20251220,PCIe-10G550X,2,2,2,7,9,7,10,Available,2025-10-17 00:00:00,No


In [22]:
filtered_df = ERP_df[ERP_df['Product Number'] == 'PA-280W-ET3'].sort_values(by='Lead Time',ascending=True)
filtered_df

,Customer,Customer PO,WO,Product Number,Qty,Stock_Available,ATP(LT),In Stock(Inventory),On Hand,On Sales Order,On PO,Component_Status,Lead Time,Picked
382,"Science Engineering Associates, Inc.",19051,SO-20251156,PA-280W-ET3,2.0,-12.0,0.0,35.0,36.0,48.0,40.0,Shortage,2025-09-17,No
668,Boston Scientific,7000607927,SO-20251219,PA-280W-ET3,1.0,-12.0,0.0,35.0,36.0,48.0,40.0,Shortage,2025-09-18,No
689,Boston Scientific,7000608058,SO-20251221,PA-280W-ET3,1.0,-12.0,0.0,35.0,36.0,48.0,40.0,Shortage,2025-09-18,Picked
551,PaR SYSTEMS LLC,450879,SO-20251184,PA-280W-ET3,1.0,-12.0,0.0,35.0,36.0,48.0,40.0,Shortage,2025-09-19,No
565,PaR SYSTEMS LLC,450879,SO-20251184,PA-280W-ET3,1.0,-12.0,0.0,35.0,36.0,48.0,40.0,Shortage,2025-09-19,No
640,Velasea,POV44892,SO-20251210,PA-280W-ET3,19.0,-12.0,0.0,35.0,36.0,48.0,40.0,Shortage,2025-09-24,No
829,Boston Scientific,7000609563,SO-20251241,PA-280W-ET3,1.0,-12.0,0.0,35.0,36.0,48.0,40.0,Shortage,2025-10-01,No
794,Boston Scientific,7000608984,SO-20251235,PA-280W-ET3,1.0,-12.0,0.0,35.0,36.0,48.0,40.0,Shortage,2025-10-01,No
694,Scout AI Inc,QTD_ScoutAI_2508085,SO-20251222,PA-280W-ET3,4.0,-12.0,0.0,35.0,36.0,48.0,40.0,Shortage,2025-10-15,No
863,Boston Scientific,7000610874,SO-20251250,PA-280W-ET3,1.0,-12.0,0.0,35.0,36.0,48.0,40.0,Shortage,2025-10-20,No


## Ready to Pick

In [34]:
# True for WOs where ALL lines have In Stock > 0 and not been picked
wo_ok = (
    ERP_df.groupby("WO")
    .apply(lambda g: (g["In Stock(Inventory)"] > g['Qty']).all() and (g["Picked"].eq("No").all()))
)


# 1) list of ready WOs
ready_wos = wo_ok[wo_ok].index.tolist()


# 2) full rows for those WOs
ready_df = ERP_df[ERP_df["WO"].isin(ready_wos)].copy()

print("Ready WOs:", ready_wos)


ready_df.to_excel(r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Output.xlsx", sheet_name="Sheet2", index=False)

Ready WOs: ['SO-20250391', 'SO-20250726', 'SO-20250728', 'SO-20251220', 'SO-20251269']


C:\Users\Admin\AppData\Local\Temp\ipykernel_26544\3256134437.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g["In Stock(Inventory)"] > g['Qty']).all() and (g["Picked"].eq("No").all()))
